In [24]:
import os
from audiolazy.lazy_midi import str2midi
from mido import Message, MetaMessage, MidiFile, MidiTrack
from mido import bpm2tempo, tempo2bpm, tick2second, second2tick

In [29]:
tests = [
    'A3_E4_A4_C#5 B3_D4_F#4_B4 A2_C#4_E4_A4 F#3_B3_F#4_B4 A3_B3_E4_B4 F#2_A3_C#4_F#4 E#3_C#4_G#4_C#5 A3_E4_C#5 B2_A3_D#4_F#4 C#3_G#3_C#4_E4 B2_D4_G#4_B4 D3_F#3_B3_F#4 D4_F#4_A4 G#3_B3_E4_B4 B2_B3_D4_E#4 B3_D#4_F#4 F#3_C#4_F#4_A4 B2_B3_D4_F#4 A#2_F#3_D4_F#4 F#3_C#4_A4 C#3_C#4_E#4_G#4 G#3_B3_F#4_B4 B2_D4_F#4_G#4 C#3_B3_D4_F#4 F#3_A#3_C#4_F#4 A#2_C#4_E4_F#4 F#3_D4_F#4_A4 B2_C#4_E4_F#4 A#2_F#3_C#4_F#4 C#3_E4_A4_C#5 C#3_A#3_C#4_F#4 E3_E4_G#4_B4 D3_B3_F#4 D3_B3_F#4 D3_B3_F#4 G#3_B3_F#4_B4 C#3_E4_A4_C#5 B2_A3_D#4_F#4 F#3_A#3_C#4_F#4 A3_B3_E4_B4 E3_E4_G#4_B4 D4_F#4_A4 G#3_B3_E4_B4 B2_A3_D#4_F#4 G#3_B3_F#4_B4 B3_D#4_F#4 B2_A3_D#4_F#4 D3_F#3_B3_F#4 A#2_C#4_E4_F#4 G#3_B3_E4_B4 D4_F#4_A4 B3_D#4_F#4 B3_D#4_F#4 F#3_C#4_F#4_A4 F#3_C#4_F#4_A4 F#3_C#4_A4 F#3_C#4_A4 C#3_C#4_E#4_G#4 C#3_C#4_E#4_G#4 B2_D4_F#4_G#4 C#3_B3_D4_F#4 F#3_C#4_A4 F#3_A#3_C#4_F#4 F#3_D4_F#4_A4 B2_A3_D#4_F#4 C#3_B3_D4_F#4 A#2_F#3_C#4_F#4 G#3_B3_E4_B4 D3_F#3_B3_F#4 B2_C#4_E4_F#4 C#3_B3_D4_F#4 B3_D#4_F#4 G#3_B3_E4_B4 F#3_D4_F#4_A4 B2_C#4_E4_F#4 C#3_E4_A4_C#5 C#3_E4_A4_C#5 C#3_A#3_C#4_F#4 E3_E4_G#4_B4 D3_B3_F#4 D3_B3_F#4 D3_B3_F#4 G#3_B3_F#4_B4 C#3_E4_A4_C#5 B2_A3_D#4_F#4 F#3_A#3_C#4_F#4 A3_B3_E4_B4 E3_E4_G#4_B4 D4_F#4_A4 G#3_B3_E4_B4 B2_A3_D#4_F#4 G#3_B3_F#4_B4 B3_D#4_F#4 B2_A3_D#4_F#4 D3_F#3_B3_F#4 A#2_C#4_E4_F#4 G#3_B3_E4_B4 D4_F#4_A4 B3_D#4_F#4 B3_D#4_F#4 F#3_C#4_F#4_A4 F#3_C#4_F#4_A4 F#3_C#4_A4 F#3_C#4_A4 C#3_C#4_E#4_G#4 C#3_C#4_E#4_G#4 B2_D4_F#4_G#4 C#3_B3_D4_F#4 F#3_C#4_A4 F#3_A#3_C#4_F#4 F#3_D4_F#4_A4 B2_A3_D#4_F#4 C#3_B3_D4_F#4 A#2_F#3_C#4_F#4 G#3_B3_E4_B4 D3_F#3_B3_F#4 B2_C#4_E4_F#4 C#3_B3_D4_F#4 B3_D#4_F#4 G#3_B3_E4_B4 F#3_D4_F#4_A4 B2_C#4_E4_F#4 C#3_E4_A4_C#5 C#3_E4_A4_C#5 C#3_A#3_C#4_F#4 E3_E4_G#4_B4 D3_B3_F#4 D3_B3_F#4 D3_B3_F#4 G#3_B3_F#4_B4 C#3_E4_A4_C#5 B2_A3_D#4_F#4 F#3_A#3_C#4_F#4 A3_B3_E4_B4 E3_E4_G#4_B4 D4_F#4_A4 G#3_B3_E4_B4 B2_A3_D#4_F#4 G#3_B3_F#4_B4 B3_D#4_F#4 B2_A3_D#4_F#4 D3_F#3_B3_F#4 A#2_C#4_E4_F#4 G#3_B3_E4_B4 D4_F#4_A4 B3_D#4_F#4 B3_D#4_F#4 F#3_C#4_F#4_A4 F#3_C#4_F#4_A4 F#3_C#4_A4 F#3_C#4_A4 C#3_C#4_E#4_G#4 C#3_C#4_E#4_G#4 B2_D4_F#4_G#4 C#3_B3_D4_F#4 F#3_C#4_A4 F#3_A#3_C#4_F#4 F#3_D4_F#4_A4 B2_A3_D#4_F#4 C#3_B3_D4_F#4 A#2_F#3_C#4_F#4 G#3_B3_E4_B4 D3_F#3_B3_F#4 B2_C#4_E4_F#4 C#3_B3_D4_F#4 B3_D#4_F#4 G#3_B3_E4_B4 F#3_D4_F#4_A4 B2_C#4_E4_F#4 C#3_E4_A4_C#5 C#3_E4_A4_C#5 C#3_A#3_C#4_F#4 E3_E4_G#4_B4 D3_B3_F#4 D3_B3_F#4 D3_B3_F#4 G#3_B3_F#4_B4 C#3_E4_A4_C#5 B2_A3_D#4_F#4 F#3_A#3_C#4_F#4 A3_B3_E4_B4 E3_E4_G#4_B4 D4_F#4_A4 G#3_B3_E4_B4 B2_A3_D#4_F#4 G#3_B3_F#4_B4 B3_D#4_F#4 B2_A3_D#4_F#4 D3_F#3_B3_F#4 A#2_C#4_E4_F#4 G#3_B3_E4_B4 D4_F#4_A4 B3_D#4_F#4 B3_D#4_F#4 F#3_C#4_F#4_A4 F#3_C#4_F#4_A4 F#3_C#4_A4 F#3_C#4_A4 C#3_C#4_E#4_G#4 C#3_C#4_E#4_G#4'
]

for i, test in enumerate(tests):
    track = chord_to_midi(test)
    store_chords(track, f'seq{i+1}.mid')
